In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import DoubleType
import os


In [2]:
spark = SparkSession.builder.appName('thuy').getOrCreate()

In [3]:
ROOT = '/home/jovyan/Thuy'
file_name = 'Table7.Budget data.csv'
Data_dir = os.path.join(ROOT, file_name)

df = spark.read.csv(Data_dir, header=True)

In [4]:
df.count()

83231

In [5]:
df= df.dropna()
df.printSchema()
df.count()

root
 |-- Production date: string (nullable = true)
 |-- Origin and Destination country: string (nullable = true)
 |-- Project Name: string (nullable = true)
 |-- Amount: string (nullable = true)



8210

In [6]:
df = df.withColumn('year', f.substring(df['Production date'], 0, 4))
df = df.withColumn('month', f.substring(df['Production date'], 6, 2))
df = df.withColumn('Origincountry', f.split(
    df['Origin and Destination country'], ":").getItem(0))
df = df.withColumn('Destinationcountry', f.split(
    df['Origin and Destination country'], ":").getItem(1))
df = df.withColumn('Year(Fiscal)', df['Project Name'].substr(
    f.locate('FY', df['Project Name'])+2, f.lit(4)))
df = df.withColumn('Month(Fiscal)', f.when(df['Project Name'].contains('Jan'),'January')
                                    .when(df['Project Name'].contains('Feb'),'February')
                                    .when(df['Project Name'].contains('Mar'),'April')
                                    .when(df['Project Name'].contains('Apr'),'January')
                                    .when(df['Project Name'].contains('May'),'May')
                                    .when(df['Project Name'].contains('Jun'),'June')
                                    .when(df['Project Name'].contains('Jul'),'July')
                                    .when(df['Project Name'].contains('Aug'),'August')
                                    .when(df['Project Name'].contains('Sep'),'September')
                                    .when(df['Project Name'].contains('Oct'),'October')
                                    .when(df['Project Name'].contains('Nov'),'November')
                                    .when(df['Project Name'].contains('Dec'),'December')) 
df = df.withColumn('BudgetAmount', f.regexp_replace(df['Amount'], ',', '.'))
df = df.withColumn('BudgetAmount', f.regexp_replace(
    df['BudgetAmount'], ' ', ''))

df = df.withColumn('FiscalYearPeriod', f.concat(df['Month(Fiscal)'], f.lit(" ") , df['Year(Fiscal)']))


In [7]:
df.show()

+---------------+------------------------------+--------------------+----------+----+-----+-------------+------------------+------------+-------------+------------+----------------+
|Production date|Origin and Destination country|        Project Name|    Amount|year|month|Origincountry|Destinationcountry|Year(Fiscal)|Month(Fiscal)|BudgetAmount|FiscalYearPeriod|
+---------------+------------------------------+--------------------+----------+----+-----+-------------+------------------+------------+-------------+------------+----------------+
|     2008年02月|               Belarus:Estonia|<Actual> May (QTD...| 10 350,00|2008|   02|      Belarus|           Estonia|        2020|          May|    10350.00|        May 2020|
|     2008年02月|                Belarus:Latvia|<Actual> June (QT...|  4 350,00|2008|   02|      Belarus|            Latvia|        2020|         June|     4350.00|       June 2020|
|     2008年02月|             Belarus:Lithuania|<Actual> July (QT...|  1 702,40|2008|   02|     

In [8]:
df = df.withColumn('BudgetAmount', df['BudgetAmount'].cast('double'))
df.printSchema()

root
 |-- Production date: string (nullable = true)
 |-- Origin and Destination country: string (nullable = true)
 |-- Project Name: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- Origincountry: string (nullable = true)
 |-- Destinationcountry: string (nullable = true)
 |-- Year(Fiscal): string (nullable = true)
 |-- Month(Fiscal): string (nullable = true)
 |-- BudgetAmount: double (nullable = true)
 |-- FiscalYearPeriod: string (nullable = true)



In [9]:
df.show()

+---------------+------------------------------+--------------------+----------+----+-----+-------------+------------------+------------+-------------+------------+----------------+
|Production date|Origin and Destination country|        Project Name|    Amount|year|month|Origincountry|Destinationcountry|Year(Fiscal)|Month(Fiscal)|BudgetAmount|FiscalYearPeriod|
+---------------+------------------------------+--------------------+----------+----+-----+-------------+------------------+------------+-------------+------------+----------------+
|     2008年02月|               Belarus:Estonia|<Actual> May (QTD...| 10 350,00|2008|   02|      Belarus|           Estonia|        2020|          May|     10350.0|        May 2020|
|     2008年02月|                Belarus:Latvia|<Actual> June (QT...|  4 350,00|2008|   02|      Belarus|            Latvia|        2020|         June|      4350.0|       June 2020|
|     2008年02月|             Belarus:Lithuania|<Actual> July (QT...|  1 702,40|2008|   02|     

In [10]:
df2 = df['year', 'month', 'Origincountry', 'Destinationcountry','BudgetAmount','FiscalYearPeriod']
df2.show()

+----+-----+-------------+------------------+------------+----------------+
|year|month|Origincountry|Destinationcountry|BudgetAmount|FiscalYearPeriod|
+----+-----+-------------+------------------+------------+----------------+
|2008|   02|      Belarus|           Estonia|     10350.0|        May 2020|
|2008|   02|      Belarus|            Latvia|      4350.0|       June 2020|
|2008|   02|      Belarus|         Lithuania|      1702.4|       July 2020|
|2008|   02|      Belarus|            Russia|      1702.4|     August 2020|
|2007|   12|       Brazil|          Paraguay|      7770.0|  September 2020|
|2008|   02|       Canada|       Netherlands|      7770.0|    October 2020|
|2007|   09|       Canada|      South Africa|    479820.0|   November 2020|
|2008|   02|       Canada|     United States|    279168.0|   December 2020|
|2009|   06|        China|             Chile|     33984.0|    January 2020|
|2009|   06|        China|              Cuba|     33739.2|   February 2020|
|2009|   05|

In [13]:
df2.count()
df2.printSchema()

root
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- Origincountry: string (nullable = true)
 |-- Destinationcountry: string (nullable = true)
 |-- BudgetAmount: double (nullable = true)
 |-- FiscalYearPeriod: string (nullable = true)



In [12]:
df2.toPandas().to_csv('budget.csv', index = None)

'3.2.1'